In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} scikit-image

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import cv2
import random
from random import seed
from random import randint
from numpy import savetxt
from PIL import Image
import os
from skimage import data, transform
from tqdm.notebook import tqdm

## Preprocess the Fashion-MNIST data

In [2]:
np.random.seed(1)
tf.random.set_seed(1)
batch_size = 128

#Load fashion MNIST and store into array training_features (dont load in labels)
(training_features, _), _ = tf.keras.datasets.fashion_mnist.load_data()
#normalize pixel values
training_features = training_features / np.max(training_features)

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(batch_size)
#dont shuffle
#training_dataset = training_dataset.shuffle(training_features.shape[0])
training_dataset = training_dataset.prefetch(batch_size * 4)

## NonRigid Transformation Function Definitions

In [3]:
# From https://mmas.github.io/linear-transformations-numpy
def linear_transformation(src, a):
    M, N = src.shape
    points = np.mgrid[0:N, 0:M].reshape((2, M*N))
    new_points = np.linalg.inv(a).dot(points).round().astype(int)
    x, y = new_points.reshape((2, M, N), order='F')
    indices = x + N*y
    return np.take(src, indices, mode='wrap')

## Generate NonRigid Images

#### Generates 60,000 Original/Transformed/Composite

In [4]:
def generateRigid():
    transforms = randint(1,7)
    pbar = tqdm(range(len(training_features)))
    for i in pbar:
        transformation = training_features[i]
        if transforms == 7:
            # translation
            x = randint(0,27)
            y = randint(0,27)
            transformation = np.roll(transformation, x, axis=0)
            transformation = np.roll(transformation, y, axis=1)
            # rotation
            degree = randint(1,360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), -np.sin(radian)],
                          [np.sin(radian), np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
            # reflection
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), np.sin(radian)],
                          [np.sin(2 * radian), -np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        elif transforms == 6:
            # translation
            x = randint(0, 27)
            y = randint(0, 27)
            transformation = np.roll(transformation, x, axis=0)
            transformation = np.roll(transformation, y, axis=1)
            # rotation
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), -np.sin(radian)],
                          [np.sin(radian), np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        elif transforms == 5:
            # translation
            x = randint(0, 27)
            y = randint(0, 27)
            transformation = np.roll(transformation, x, axis=0)
            transformation = np.roll(transformation, y, axis=1)
            # reflection
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), np.sin(radian)],
                          [np.sin(2 * radian), -np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        elif transforms == 4:
            # rotation
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), -np.sin(radian)],
                          [np.sin(radian), np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
            # reflection
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), np.sin(radian)],
                          [np.sin(2 * radian), -np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        elif transforms == 3:
            # translation
            x = randint(0, 27)
            y = randint(0, 27)
            transformation = np.roll(transformation, x, axis=0)
            transformation = np.roll(transformation, y, axis=1)
        elif transforms == 2:
            # rotation
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), -np.sin(radian)],
                          [np.sin(radian), np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        else:
            # reflection
            degree = randint(1, 360)
            radian = np.radians(degree)
            a = np.array([[np.cos(radian), np.sin(radian)],
                          [np.sin(2 * radian), -np.cos(radian)]])
            transformation = linear_transformation(transformation, a)
        
        composite = (training_features[i]/2) + (transformation/2)
        
        #create image objects of transformation/original/composite
        transformation_img = Image.fromarray(np.uint8(transformation*255),'L')
        original_img = Image.fromarray(np.uint8(training_features[i]*255),'L')
        composite_img = Image.fromarray(np.uint8(composite*255),'L')
        
        #save images to disk
        transformation_img.save('C:\\Users\\gunne\\Desktop\\CS791\\Rigid\\Transformed\\'+str(i)+'.jpg')
        original_img.save('C:\\Users\\gunne\\Desktop\\CS791\\Rigid\\Original\\'+str(i)+'.jpg')
        composite_img.save('C:\\Users\\gunne\\Desktop\\CS791\\Rigid\\Composite\\'+str(i)+'.jpg')


if __name__ == "__main__":
    generateRigid()